# **INGESTION DE "DEVICE.CSV"**

##### **1.- Leer el archivo usando "DataFrameReader" de spark**

In [2]:
#p_file_date = "2025-07-07"
#p_environment = "Development"

StatementMeta(, cd3b0e1a-be08-4352-be03-5d80f86f184e, 3, Finished, Available, Finished)

In [ ]:
%run configuration

StatementMeta(, , -1, Cancelled, , Cancelled)

In [ ]:
%run common_functions

StatementMeta(, , -1, Cancelled, , Cancelled)

In [ ]:
#Importaciones de los tipos de datos para definir el esquema
from pyspark.sql.types import StructType, StructField, IntegerType, DateType

StatementMeta(, , -1, Cancelled, , Cancelled)

In [ ]:
# Esta es la estructura del df, esto se define para que inferSchema no haga un proceso extra 
# sirve para grandes volumenes de datos, asi se demoran menos en cargar
device_schema = StructType ( fields = [
    StructField("DeviceID", IntegerType(), False),
    StructField("BrandID", IntegerType(), True),
    StructField("ModelID", IntegerType(), True),
    StructField("DisplayID", IntegerType(), True),
    StructField("CameraID", IntegerType(), True),
    StructField("ConnectivityID", IntegerType(), True),
    StructField("OSID", IntegerType(), True),
    StructField("PhysicalSpecID", IntegerType(), True),
    StructField("ReleasedYear", IntegerType(), True),
    StructField("ReleasedAnnounced", DateType(), True),
])

StatementMeta(, , -1, Cancelled, , Cancelled)

In [ ]:
# Ruta del archivo CSV en Azure Blob Storage o ADLS
abfs_path = f"{bronze_folder_path}/{p_file_date}/device.csv"

# Leer el archivo CSV desde ABFS
df = spark.read.option("header", True) \
                .schema(device_schema) \
                .csv(abfs_path)

StatementMeta(, , -1, Cancelled, , Cancelled)

In [ ]:
# Revisar esquema, en DeviceID parece que si admite nulos, pero internamente no es asi
# ya que lo especificamos en el esquema manualmente
df.printSchema()

StatementMeta(, , -1, Cancelled, , Cancelled)

##### **2.- Borrar columna "ReleasedYear"**

In [ ]:
# Borrar algunas clumnas de df, tambien se puede usar select para solo dejar algunas pocas
df2 = df.drop("ReleasedYear")

StatementMeta(, , -1, Cancelled, , Cancelled)

##### **3.- Cambiar el nombre de las columnas según lo "requerido"**

In [ ]:
new_cols = ["device_id", "brand_id", "model_id", "display_id", "camera_id", "connectivity_id", "os_id", "physical_spec_id", "released_announced"]

df_renamed = df2.toDF(*new_cols)

StatementMeta(, , -1, Cancelled, , Cancelled)

##### **4.- Agregar columnas "ingestion_date" y "environment"**

In [ ]:
from pyspark.sql.functions import lit

df4 = add_ingestion_date(df_renamed) \
        .withColumn("environment", lit(p_environment)) \
        .withColumn("file_date", lit(p_file_date))

StatementMeta(, , -1, Cancelled, , Cancelled)

In [ ]:
%%sql
DROP TABLE if EXISTS device

StatementMeta(, , -1, Cancelled, , Cancelled)

##### **5.- Escribir datos en el "lakehouse" en formato delta**

In [ ]:
df4.write.format("delta").mode("overwrite").saveAsTable("device")

StatementMeta(, , -1, Cancelled, , Cancelled)

In [ ]:
%%sql
SELECT * from device
limit(10)

StatementMeta(, , -1, Cancelled, , Cancelled)